In [36]:
import pandas as pd
import json
from textblob import TextBlob
import operator
import re

In [37]:
# import sys
# !{sys.executable} - m pip install textblob

In [38]:
df = pd.read_csv('data/DomoArigatoData.csv') 

In [39]:
df

,Pathogenic?,Gene,history_class,full_history
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis..."
...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...


In [40]:
keylist = []
for index, row in pd.DataFrame(df['full_history']).itertuples():
    split_arr = row.split(" ; ")
    for keyval in split_arr:
        if keyval.count(':') == 1:
            if keyval.split(':')[0] not in keylist:
                keylist.append(keyval.split(':')[0])
        else:
            print(index, row)
#     for keyval in split_arr:
#         if 

In [41]:
keylist

['consent_approval',
 'ethnicity',
 'cancer_dx',
 'cancer_dx_type',
 'cancer_dx_age',
 'known_brca',
 'known_cancer',
 'rel_relation',
 'rel_cancer',
 'rel_age',
 '_method',
 'other_cancer',
 'relationships']

In [42]:
for key in keylist:
    df[key] = ""

In [43]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,known_cancer,rel_relation,rel_cancer,rel_age,_method,other_cancer,relationships
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,,,,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,,,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,,,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,,,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,,,,,,,,,,,,,


In [44]:
for index, value in pd.DataFrame(df['full_history']).itertuples():
    split_arr = value.split(" ; ")
    joe = [{x.split(":")[0]: x.split(":")[1]} for x in split_arr]
    for keyval in joe:
        col_name = list(keyval.keys())[0]
        val = list(keyval.values())[0]
        df.at[index, col_name] = val
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,known_cancer,rel_relation,rel_cancer,rel_age,_method,other_cancer,relationships
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,yes,"[""Mother"", ""Grandmother Mothers Side"", ""Grandf...","[""Ovarian"", ""Leukemia CML"", ""Bladder "", ""Squam...","[""74"", ""82"", ""35"", ""67"", ""55"", ""64"", ""55""]",,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,yes,"[""Father"", ""Sister"", ""Aunt""]","[""Colon"", ""Uterine "", ""Colon""]","[""74"", ""60"", ""70""]",,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,yes,"[""father"", ""uncle"", """"]","[""he died of lymphoma at 86"", ""prostae"", """"]","[""72"", ""not sure"", """"]",,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,yes,"[""brother"", ""brother"", ""female cousin"", ""mater...","[""colon"", ""colon"", ""colon"", ""breast"", ""brain"",...","[""60"", ""65"", ""72"", ""80"", ""70"", ""74""]",,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,no,"[""Grandfather"", """", """"]","[""Lymph node"", """", """"]","[""81"", """", """"]",patch,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,no,"["""", """", """"]","["""", """", """"]","["""", """", """"]",patch,,


In [45]:
rel_relation_list = []
for index, row in pd.DataFrame(df['rel_relation']).itertuples():
    try:
        for x in json.loads(row):
            rel_relation_list.append(x.lower())
    except:
        print(index, row)

419 
438 
439 
440 
1252 
1253 
1269 
1270 
1271 
1272 
1273 
1509 
1544 
2002 
2200 


In [46]:
rel_relation_list = list(set(list(filter(None, rel_relation_list))))

In [47]:
for i in range(len(rel_relation_list)):
    rel_relation_list[i]=TextBlob(rel_relation_list[i]).correct()

In [48]:
rel_relation_list = [str(x) for x in rel_relation_list]

In [49]:
for index, row in df.loc[df['relationships'] != ""].iterrows():
    parsed = row['relationships'].replace('=>', ':')
    box = json.loads(parsed)
    
    for i in range(len(box)):
        for item in box[i].items():
            if item[0]=='relationship_type':
                df.at[index, 'rel_relation'] = f'["{item[1]}"]'
            if item[0]=='cancer_dx_type':
                df.at[index, 'rel_cancer'] = f'["{item[1]}"]'
            if item[0]=='cancer_dx_age':
                df.at[index, 'rel_age'] = f'["{item[1]}"]'

In [50]:
for y in range(1, 4):
    df[f"cancer_dx_type_{str(y)}"] = ""
    df[f"cancer_dx_age_{str(y)}"] = ""
for i in range(1, 24):
    df[f"rel_{str(i)}_relationship"] = ""
    for t in range(1, 11):
        df[f"rel_{str(i)}_cancer_{str(t)}"] = ""
    for u in range(1, 4):
        df[f"rel_{str(i)}_age_{str(u)}"] = ""

In [51]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [52]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [53]:
rel_relation_search_list = ["mother","father","nephew","niece","uncle","aunt","brother","sister","grandmother","grandfather","great grandfather","great grandmother","cousin","dad","daughter","son","grand aunt","grand uncle","mom","great aunt","great uncle", "grandpa", "grandma"]
def find_probable_relation(test):
    test = test.lower()
    test = re.sub("([\(\[]).*?([\)\]])", "", test)
    if "side" in test:
        test = re.sub("(\w*'*\w)(\sside)", "", test)
    if "maternal" in test:
        test = test.replace("maternal", "")
    if "paternal" in test:
        test = test.replace("paternal", "")
    test = test.replace(" ", "")
    scores = {}
    for rel in rel_relation_search_list:
        scores[rel] = 1 - levenshteinDistance(test, rel)
    guess_relation = relation_dict[max(scores.items(), key=operator.itemgetter(1))[0]]
    df.at[index, f"rel_{i+1}_relationship"] = guess_relation

In [54]:
relation_dict = {
    'mother': 'mother',
    'mom': 'mother',
    'father': 'father',
    'dad': 'dad',
    'nephew': 'nephew',
    'niece': 'niece',
    'uncle': 'uncle',
    'aunt': 'aunt',
    'brother': 'brother',
    'sister': 'sister',
    'cousin': 'cousin',
    'grandmother': 'grandmother',
    'grandfather': 'grandfather',
    'great grandfather': 'great grandfather',
    'great grandmother': 'great grandmother',
    'daughter': 'daughter',
    'son': 'son',
    'grand aunt': 'great aunt',
    'grand uncle': 'great uncle',
    'great aunt': 'great aunt',
    'great uncle': 'great uncle',
    'grandpa': 'grandfather',
    'grandma': 'grandmother'
}

In [55]:
def find_probable_age(test, index, rel_index):
    test = re.findall("[0-9]+", test)
    for i in range(len(test)):
        df.at[index, f"rel_{rel_index+1}_age_{i+1}"] = test[i]
#     if len(test) == 0:
#         pass
#     elif len(test) == 1: 
#         df.at[index, f"rel_{rel_index+1}_age_1"] = test[0]
#     else:
#         for i in range(len(test)):
#             df.at[index, f"rel_{rel_index+1}_age_{i+1}"] = test[i]

In [56]:
rel_cancer_search_list = ['vulvar','lymphoma','ovarian','brain','breast','prostate','blood','bone','abdominal','leukemia','lung','ampulla','skin','anal','colon','thyroid','appendix','lympnodes','basal','bile duct','bladder','intestines','liver','bowel','sinus','spinal','uterine','cervical','pancreas','cheek','ciliary','voice box','endometrial','rectum','tonsil','tongue','esophageal','thrombocytopenia','testicles','stomach','squamous','throat','retina','renal','myeloma', 'melanoma', 'oral','muscle','polyps','kidney','gall bladder','jaw','heart','glaucoma','glioblastoma','duodenum']

In [57]:
def find_probable_cancer(test, index, rel_index):
    test = test.lower()
    if test == "?" or test == "unknown":
        pass
    else:
        test = re.split(",|/|;", test)
        test = [re.sub("([\(\[]).*?([\)\]])", "", x) for x in test]
        for i in range(len(test)):
            scores = {}
            for rel in rel_cancer_search_list:
                scores[rel] = 1 - levenshteinDistance(test[i], rel)
            guess_cancer = max(scores.items(), key=operator.itemgetter(1))[0]
            df.at[index, f"rel_{rel_index+1}_cancer_{i+1}"] = guess_cancer

In [58]:
for index, row in df[['rel_relation', 'rel_cancer', 'rel_age']].iterrows():
    relations = []
    cancers = []
    ages = []
    if row['rel_relation'] != "":
        relations = json.loads(row['rel_relation'])
    else:
        pass
    if row['rel_cancer'] != "":
        cancers = json.loads(row['rel_cancer'])
    else:
        pass
    if row['rel_age'] != "":
        ages = json.loads(row['rel_age'])
    else:
        pass
    
    for i in range(len(relations)):
        if relations[i] == "":
            pass
        else:
            guess_relation = find_probable_relation(relations[i])
            guess_cancer = find_probable_cancer(cancers[i], index, i)
            guess_age = find_probable_age(ages[i], index, i)


In [59]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [60]:
for index, row in df.loc[df['cancer_dx_type'] != ""].iterrows():
    test = row['cancer_dx_type'].lower()
    test = test.replace("precancer", "")
    test = test.replace("pre-cancer", "")
    if test == "na" or test == "n/a":
        pass
    else:
        test = test.split(',')
        for i in range(len(test)):
            scores = {}
            for rel in rel_cancer_search_list:
                scores[rel] = 1 - levenshteinDistance(test[i], rel)
            guess_cancer = max(scores.items(), key=operator.itemgetter(1))[0]
            df.at[index, f"cancer_dx_type_{i+1}"] = guess_cancer

In [61]:
for index, row in df.loc[df['cancer_dx_age'] != ""].iterrows():
    test = re.findall("[0-9]+", row['cancer_dx_age'])
    for i in range(len(test)):
        df.at[index, f"cancer_dx_age_{i+1}"] = test[i]

In [62]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,english extraction caucasion,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,"white, english, irish",yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,"Caucasian, some Native American",yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Caucasian,no,,,no,...,,,,,,,,,,


In [63]:
# df2 = pd.read_csv('data/Ethnicity.csv')
# adjectives = [x.replace(u'\xa0', u'').lower() for x in list(df2['Adjective'])]
# race = [x.replace(u'\xa0', u'') for x in list(df2['Race'])]
# final = []
# final.extend([list(a) for a in zip(adjectives, race)])
# ethinicities = dict(final)
# data = [TextBlob(x.lower().strip().replace('','')).correct() for x in list(df['ethnicity'])]

for i in range(len(data)):
    scores = {}
    for rel in list(ethinicities.keys()):
            scores[rel] = 1 - levenshteinDistance(data[i], rel)
    ethnicity_guess = ethinicities[max(scores.items(), key=operator.itemgetter(1))[0]]
    df.at[i, 'ethnicity'] = ethnicity_guess

In [64]:
df

,Pathogenic?,Gene,history_class,full_history,consent_approval,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,NaN,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,yes,White,yes,Pre-Cancer Vulvar Due to HPV,57,unknown,...,,,,,,,,,,
1,False,NaN,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,,White,no,Melanoma,32,unknown,...,,,,,,,,,,
2,False,NaN,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,,Hispanic,yes,Breast,37,unknown,...,,,,,,,,,,
3,False,NaN,strong_personal,ethnicity:english extraction caucasion ; cance...,,White,yes,Prostate,55,unknown,...,,,,,,,,,,
4,False,NaN,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",yes,White,yes,uterine,58,unknown,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2209,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,unknown,...,,,,,,,,,,
2210,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,Native American,yes,Cervical carcinoma in situ,22,unknown,...,,,,,,,,,,
2211,False,NaN,none,_method:patch ; consent_approval:yes ; ethnici...,yes,White,no,,,no,...,,,,,,,,,,


In [65]:
df['Gene'] = df['Gene'].fillna('')
df['history_class'] = df['history_class'].replace('not_strong ', 'not_strong')
df = df.drop(columns=['consent_approval'])
df['cancer_dx']=df['cancer_dx'].replace('Yes','yes').replace('No', 'no').map(dict(yes=1, no=0))
df[df['cancer_dx'].isnull()]['cancer_dx'][399]=1
df['cancer_dx']=df['cancer_dx'].fillna(0)
df = df.drop(columns=['_method'])
df = df.drop(columns=['known_cancer'])
df['known_brca']=df['known_brca'].fillna('unknown')
df['known_brca']=df['known_brca'].replace('Yes','yes').replace('No', 'no').replace('U', 'unknown').map(dict(yes=1, no=0, unknown=-1))

In [66]:
df

,Pathogenic?,Gene,history_class,full_history,ethnicity,cancer_dx,cancer_dx_type,cancer_dx_age,known_brca,rel_relation,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,False,,strong_personal,consent_approval:yes ; ethnicity:White ; cance...,White,1.0,Pre-Cancer Vulvar Due to HPV,57,-1.0,"[""Mother"", ""Grandmother Mothers Side"", ""Grandf...",...,,,,,,,,,,
1,False,,strong_personal,ethnicity:White ; cancer_dx:no ; cancer_dx_typ...,White,0.0,Melanoma,32,-1.0,"["""", """", """"]",...,,,,,,,,,,
2,False,,strong_personal,ethnicity:Hispanic ; cancer_dx:yes ; cancer_dx...,Hispanic,1.0,Breast,37,-1.0,"[""Father"", ""Sister"", ""Aunt""]",...,,,,,,,,,,
3,False,,strong_personal,ethnicity:english extraction caucasion ; cance...,White,1.0,Prostate,55,-1.0,"[""father"", ""uncle"", """"]",...,,,,,,,,,,
4,False,,strong_personal,"consent_approval:yes ; ethnicity:white, englis...",White,1.0,uterine,58,-1.0,"[""brother"", ""brother"", ""female cousin"", ""mater...",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2208,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,-1.0,"["""", """", """"]",...,,,,,,,,,,
2209,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,-1.0,"[""Grandfather"", """", """"]",...,,,,,,,,,,
2210,False,,none,_method:patch ; consent_approval:yes ; ethnici...,Native American,1.0,Cervical carcinoma in situ,22,-1.0,"["""", """", """"]",...,,,,,,,,,,
2211,False,,none,_method:patch ; consent_approval:yes ; ethnici...,White,0.0,,,0.0,"["""", """", """"]",...,,,,,,,,,,


In [67]:
df.to_csv('data/cleaned.csv', index = False)